# Init

In [1]:
# Import utils
import numpy as np
import pandas as pd
import math
import time
import json
import pyreadr
import pickle
from joblib import dump, load
import random
import string
import os
import copy
import datetime
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
# Generate a random key
def get_random_key(l):
    
    """
    
    Generates random key of letters and numbers where l defines the desired length.
    
    """
    
    elements = string.ascii_letters + string.digits
    key = ''.join(random.choice(elements) for i in range(l))
    
    return key

# Pre-process raw data

The data consists of the following information:

* **Patient**: id, age, sex, health status, ...

* **Treatment**: duration, date, area, treatment details (DVGS codes, codes for used endoscopes, indicators for emergency, stay/ambulant treatemnts), ...

* **Surgeons**: information on surgeons, assistants, and nurses that conducted the treatment

Each row in the data belongs to one surgery / treatment / examination.

In [3]:
# Load raw data
data = pd.read_csv('/home/fesc/dddex/PatientScheduling/Data/dataHCScheduling2.csv', sep=';')

In [4]:
# Format dates and add columns for year, month, ...
for r in range(len(data)):
    
    date = [int(i) for i in reversed(data['Datum'].iloc[r].split('.'))]

    data.loc[r, 'year'] = 2000+date[0]
    data.loc[r, 'month'] = date[1]
    data.loc[r, 'week'] = datetime.date(year=2000+date[0], month=date[1], day=date[2]).isocalendar()[1]
    data.loc[r, 'day'] = date[2]
    data.loc[r, 'weekday'] = datetime.date(year=2000+date[0], month=date[1], day=date[2]).weekday()
    data.loc[r, 'date'] = datetime.date(year=2000+date[0], month=date[1], day=date[2])

In [5]:
# 1-hot encode weekday
enc = OneHotEncoder()
enc = enc.fit(np.array(data['weekday']).reshape(-1,1))
weekday = enc.transform(np.array(data['weekday']).reshape(-1,1)).toarray()
cols = enc.get_feature_names_out(['weekday'])
data[cols] = weekday

In [6]:
# Add areas
areas = [
    
    'Bereich_Bronchoskopie', # bronchoscopy
    'Bereich_ERCP',          # endoscopic retrograde cholangiopancreatography
    'Bereich_Endosonographie', # endosonography
    'Bereich_Gastroskopie', # gastroscopy
    'Bereich_Koloskopie' # colonoscopy
    
]

area = [areas[i] for i in np.where(np.array(data[areas]) == 1)[1]]
data['area'] = area

In [7]:
# 1-hot encode type of hospitalization ('Aufenthalt')
data = data.rename(columns={'Aufenthalt': 'hospitalization'})
enc = OneHotEncoder()
enc = enc.fit(np.array(data['hospitalization']).reshape(-1,1))
hospitalization = enc.transform(np.array(data['hospitalization']).reshape(-1,1)).toarray()
cols = enc.get_feature_names_out(['hospitalization'])
data[cols] = hospitalization

In [8]:
# 1-hot encode health status (ASA code)
data = data.rename(columns={'ASA': 'health_status_code'})
enc = OneHotEncoder()
enc = enc.fit(np.array(data['health_status_code']).reshape(-1,1))
health_status_code = enc.transform(np.array(data['health_status_code']).reshape(-1,1)).toarray()
cols = enc.get_feature_names_out(['health_status_code'])
data[cols] = health_status_code

In [9]:
# Team setup
surgeons = [

    'Brand', 'Bumm', 'Dick', 'Döring', 'Engel', 'Flemming', 'Flüchter', 'Geier',
    'Gernert', 'Groneberg', 'Götze', 'Hann', 'Hartlapp', 'Hose', 'Jung', 'Kienle',
    'Koscher', 'Kudlich', 'Köhler', 'Maniuc', 'Matthes', 'Megerle', 'Meining',
    'Meir', 'Mersi', 'Messerschmidt', 'Petri', 'Plaßmeiner', 'Reibetanz', 'Reimer',
    'Scheurlen', 'Schoettker', 'Schweinfurth', 'Stationsarzt Neuologie', 'Steinhardt',
    'Stier', 'Weich', 'Weiß', 'Wierlemann', 'Yilmaz'
    
]

assistants = [

    'Akpinar', 'Boehn', 'Endres', 'Estenfelder', 'Hospitant', 'Lau', 'Mennig', 'Nöth',
    'OTA', 'Pfützner', 'Purrer', 'Saam', 'Seelbach', 'Smol', 'Streit', 'Thiergärtner',
    'Uhlein', 'Wiesemann'
]

# Clean erratic data

In [10]:
# Remove erratic dates (there are 2 erratic dates in 2020)
data = data.loc[data['date'] <= datetime.date(2019, 12, 31)]

In [11]:
# Remove NaN in sex
data = data.loc[~np.isnan(data.Geschlecht)]

# Sanetize data

In [12]:
# Surgeon names
surgeons_sanetized = dict([(surgeon, 'surgeon_'+get_random_key(16))
                           for (surgeon, i) in zip(surgeons, range(len(surgeons)))])

check = len(set(surgeons_sanetized.values())) == len(surgeons)
print('Random key generation worked:', check)

data = data.rename(columns=surgeons_sanetized)

Random key generation worked: True


In [13]:
# Assistant names
assistants_sanetized = dict([(assistant, 'assistant_'+get_random_key(16))
                             for (assistant, i) in zip(assistants, range(len(assistants)))])

check = len(set(assistants_sanetized.values())) == len(assistants)
print('Random key generation worked:', check)

data = data.rename(columns=assistants_sanetized)

Random key generation worked: True


In [14]:
# Area names
areas_sanetized = dict([(area, 'area_'+get_random_key(16)) 
                        for (area, i) in zip(areas, range(len(areas)))])

check = len(set(areas_sanetized.values())) == len(areas)
print('Random key generation worked:', check)

data = data.rename(columns=areas_sanetized)
data['area'] = data['area'].replace(areas_sanetized, regex=True)

Random key generation worked: True


In [15]:
# Endoscope codes
endoscope_codes = [
    
    '1U047K185','2C738K060','2G411K040','3U048K046','5G402K039',
    'A120031','A120183','A120762','A121432','CFH180AL','CFHQ190AL',
    'CFHQ190L','CFQ160','CFQ160AI','CFQ160AL','CFQ180AL','CFUE190',
    'EC760PV','EG34J10U','EG3670URK','EG36J10UR','EG3870UTK','EG38J10UT',
    'EG580UR','EG580UT','EG760CT','EG760R','EGUTK','G120515','G120560',
    'G3870','GCUCT180','GFUCT180','GIF140','GIF180','GIF1T140',
    'GIF1TH190','GIF2T160','GIFH180','GIFH190','GIFHQ190','GIFHQ190L',
    'GIFQ140','GIFQ160','GIFQ165','GIFQ180','GIFXP','GIFXP160','GIFXP180',
    'GIFXP180N','GIFXP190N','GIFXP','GITTQ160','GIVEN','H121383',
    'H180','HQ190','IFQ180','JF140R','K120064', 'PCF140L','PCFH180AL',
    'PCFH190','Q180','SB3','SIFQ180','Spyglass', 'TJF160R','TJF160VR',
    'TJFQ180V','TL2900156'

]

endoscope_codes = [endoscope_code for endoscope_code in endoscope_codes if endoscope_code in data.columns]

endoscope_codes_sanetized = dict([(endoscope_code, 'endoscope_code_'+get_random_key(16))
                                 for (endoscope_code, i) in zip(endoscope_codes, range(len(endoscope_codes)))])

check = len(set(endoscope_codes_sanetized.values())) == len(endoscope_codes)
print('Random key generation worked:', check)

data = data.rename(columns=endoscope_codes_sanetized)

Random key generation worked: True


In [16]:
# Examination codes
examination_codes = [

    'ERCP010', 'ERCP020', 'ERCP030', 'ERCP040', 'ERCP050', 'ERCP060', 'ERCP070', 'ERCP080',
    'ERCP090', 'ERCP100', 'ERCP110', 'ERCP120', 'ERCP130', 'ERCP140',
    'EUSOGT010', 'EUSOGT020', 'EUSOGT030', 'EUSUGT010', 'EUSUGT020',
    'INTESTO010', 'INTESTO040',
    'KOLO010', 'KOLO020', 'KOLO030', 'KOLO040', 'KOLO050', 'KOLO060', 'KOLO070', 'KOLO080',
    'KOLO090', 'KOLO100', 'KOLO110', 'KOLO120', 'KOLO130', 'KOLO150', 'KOLO170',
    'PEG-Kontrolle',
    'PROKTO020',
    'SPEZ010',
    'ÖGD010', 'ÖGD020', 'ÖGD030', 'ÖGD040', 'ÖGD050', 'ÖGD060', 'ÖGD070', 'ÖGD080', 'ÖGD090',
    'ÖGD100', 'ÖGD110', 'ÖGD120', 'ÖGD130', 'ÖGD140', 'ÖGD150', 'ÖGD160', 'ÖGD170', 'ÖGD180',
    'ÖGD190', 'ÖGD200', 'ÖGD210', 'ÖGD220', 'ÖGD230', 'ÖGD240', 'ÖGD250'

]

examination_codes = [examination_code for 
                     examination_code in examination_codes 
                     if examination_code not in endoscope_codes]

examination_codes_sanetized = dict([(examination_code, 'examination_code_'+get_random_key(16))
                                 for (examination_code, i) in zip(examination_codes, range(len(examination_codes)))])

check = len(set(examination_codes_sanetized.values())) == len(examination_codes)
print('Random key generation worked:', check)

data = data.rename(columns=examination_codes_sanetized)

Random key generation worked: True


In [17]:
# Month
month_codes = [col for col in data.columns if 'Monat' in col]

month_codes_sanetized = dict(zip(month_codes, ['month_'+get_random_key(16) 
                                               for month_code in month_codes]))

check = len(set(month_codes_sanetized.values())) == len(month_codes)
print('Random key generation worked:', check)

data = data.rename(columns=month_codes_sanetized)

Random key generation worked: True


In [18]:
# Weekday
weekday_codes = [col for col in data.columns if 'weekday_' in col]

weekday_codes_sanetized = dict(zip(weekday_codes, ['weekday_'+get_random_key(16) 
                                                   for weekday_code in weekday_codes]))

check = len(set(weekday_codes_sanetized.values())) == len(weekday_codes)
print('Random key generation worked:', check)

data = data.rename(columns=weekday_codes_sanetized)

Random key generation worked: True


In [19]:
# Date
dates = sorted(list(set(data['date'])))
dates = pd.DataFrame.from_dict(dict([(date, i+1) for (date, i) in zip(dates, range(len(dates)))]), 
                               orient='index', columns=['i'])
dates = dates.reset_index().rename(columns={'index': 'date'})

data = pd.merge(left=data,
        right=dates,
        on = 'date')
data = data.drop(columns='date')
data = data.rename(columns={'i': 'date'})

In [20]:
# Patient
patients = list(set(data['patient_id']))

patients_sanetized = dict([(patient, 'patient_'+get_random_key(32)) 
                           for (patient, i) in zip(patients, range(len(patients)))])

check = len(set(patients_sanetized.values())) == len(patients)
print('Random key generation worked:', check)

data['patient_id'] = data['patient_id'].replace(patients_sanetized, regex=True)

Random key generation worked: True


In [21]:
# Health status
health_status_codes = [col for col in data.columns if 'health_status_code_' in col]

health_status_codes_sanetized = dict(zip(health_status_codes, ['health_status_'+get_random_key(16) 
                                               for health_status_code in health_status_codes]))

check = len(set(health_status_codes_sanetized.values())) == len(health_status_codes)
print('Random key generation worked:', check)

data = data.rename(columns=health_status_codes_sanetized)

Random key generation worked: True


In [22]:
# Hospitalization
hospitalization_codes = [col for col in data.columns if 'hospitalization_' in col]

hospitalization_codes_sanetized = dict(zip(hospitalization_codes, ['hospitalization_'+get_random_key(16) 
                                               for hospitalization_code in hospitalization_codes]))

check = len(set(hospitalization_codes_sanetized.values())) == len(hospitalization_codes)
print('Random key generation worked:', check)

data = data.rename(columns=hospitalization_codes_sanetized)

Random key generation worked: True


In [23]:
# Rename columns to english
data = data.rename(columns={
    
    'Notfall': 'emergency',
    'Beratung': 'consultation',
    'Alter': 'age',
    'Geschlecht': 'sex',
    'Dauer': 'duration'

})

In [24]:
# Columns to be removed
remove = [
    
    'year',
    'month',
    'week',
    'weekday',
    'day',
    'Datum',
    'hospitalization',
    'health_status_code'

]

data = data.drop(columns=remove)

# Save

In [25]:
# See all columns
set(data.columns)

{'age',
 'area',
 'area_9BjwrnFuXmkSUqNd',
 'area_G1tK48shDUB3Jm8n',
 'area_WVLukn2XWHS6OyQZ',
 'area_Y6F5RxDwY1dwMg8j',
 'area_pxfcVXf6XvBT0sZD',
 'assistant_1DzIl8Gj8OURXBUl',
 'assistant_4d3oCQJFFnhFuyuW',
 'assistant_4d5eJ5VclUXVaKaw',
 'assistant_6ADM7UQdhVW6vIJR',
 'assistant_E3IeIwEnec1cXyby',
 'assistant_GleWV5QHmlmGUeV5',
 'assistant_KAvUKX8MBxkgQ2fc',
 'assistant_L64DwW6iStCXilUI',
 'assistant_NN2mlrEM2e8cNpww',
 'assistant_Ojae1XfvqfMXnOI8',
 'assistant_Q9BiXmnpX0KoVmKU',
 'assistant_TuU5L75bxBolF50S',
 'assistant_V5SA1zKn1Z6Tl9Ng',
 'assistant_ZFjJ0p9TeEDy8ZjP',
 'assistant_bNAcOudnEl5DKgo5',
 'assistant_wZdbtLC5OK5mFpqn',
 'assistant_xbJhvXrXQDIHw88B',
 'assistant_yn5KXkyhzX3Ynz8T',
 'consultation',
 'date',
 'duration',
 'emergency',
 'endoscope_code_253UureneBA1YA5e',
 'endoscope_code_48M3wyaOI7jjaQoJ',
 'endoscope_code_87LonqHlo4WwbK9O',
 'endoscope_code_BC2sz3pOsb2LS78A',
 'endoscope_code_BSI449uODTGy7ST0',
 'endoscope_code_De8E4JOL75pAIX6A',
 'endoscope_code_JEhe2zwZc

In [26]:
# Save
data.to_csv('/home/fesc/dddex/PatientScheduling/Data/data.csv', sep=',', index=False)